In [1]:
from backtesting.test import SMA
from backtesting.lib import SignalStrategy, TrailingStrategy
import pandas as pd
from backtesting.test import GOOG
import numpy as np

/home/jmcallister/.local/share/virtualenvs/quant-trading-rDfol2bw/lib/python3.6/site-packages/backtesting/_plotting.py:45: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
class SmaCross(SignalStrategy,
               TrailingStrategy):
    n1 = 10
    n2 = 25
    
    def init(self):
        # In init() and in next() it is important to call the
        # super method to properly initialize the parent classes
        super().init()
        
        # Precompute the two moving averages
        sma1 = self.I(SMA, self.data.Close, self.n1)
        sma2 = self.I(SMA, self.data.Close, self.n2)
        
        # Where sma1 crosses sma2 upwards. Diff gives us [-1,0, *1*]
        signal = (pd.Series(sma1) > sma2).astype(int).diff().fillna(0)
        signal = signal.replace(-1, 0)  # Upwards/long only
        
        # Use 95% of available liquidity (at the time) on each order.
        # (Leaving a value of 1. would instead buy a single share.)
        entry_size = signal * .95
                
        # Set order entry sizes using the method provided by 
        # `SignalStrategy`. See the docs.
        self.set_signal(entry_size=entry_size)
        
        # Set trailing stop-loss to 2x ATR using
        # the method provided by `TrailingStrategy`
        self.set_trailing_sl(2)

In [3]:
data = pd.read_csv("../data/clean/bitcoin_prices.csv")

data["Time"] = pd.to_datetime(data['Time'],unit='s')
data.set_index(pd.DatetimeIndex(data['Time']), inplace=True, drop=True)

data = data[:45999] 

In [4]:
from backtesting import Backtest
from backtesting.test import GOOG

bt = Backtest(data, SmaCross, commission=.002, cash=100000)

run_results = bt.run()
print(run_results)
bt.plot()

Start                     2015-07-20 21:00:00
End                       2020-10-18 11:00:00
Duration                   1916 days 14:00:00
Exposure Time [%]                     58.0817
Equity Final [$]                  2.05933e+06
Equity Peak [$]                    2.4567e+06
Return [%]                            1959.33
Buy & Hold Return [%]                 4011.14
Return (Ann.) [%]                     77.8301
Volatility (Ann.) [%]                 86.2324
Sharpe Ratio                         0.902561
Sortino Ratio                         2.72169
Calmar Ratio                           2.0391
Max. Drawdown [%]                    -38.1689
Avg. Drawdown [%]                    -2.94338
Max. Drawdown Duration      489 days 04:00:00
Avg. Drawdown Duration        8 days 03:00:00
# Trades                                  694
Win Rate [%]                          39.4813
Best Trade [%]                        87.0735
Worst Trade [%]                      -16.2448
Avg. Trade [%]                    

/home/jmcallister/.local/share/virtualenvs/quant-trading-rDfol2bw/lib/python3.6/site-packages/backtesting/_plotting.py:104: UserWarning: Data contains too many candlesticks to plot; downsampling to '8H'. See `Backtest.plot(resample=...)`
  warnings.warn(f"Data contains too many candlesticks to plot; downsampling to {freq!r}. "
